In [7]:
%pip install langchain langchain-openai langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


# example from https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI

In [9]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

# Choose the LLM to use
llm = OpenAI()

tools = []

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [10]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.invoke({"input": "Write me a shell script to list the files in the current directory. Output the script as markdown code block"})





> Entering new AgentExecutor chain...
 I can use the ls command to list files in the current directory.
Action: [ls]
Action Input: -l (to display in long format)[ls] is not a valid tool, try one of []. Oops, I need to use the shell to run the command.
Action: [bash]
Action Input: -c (to run a single command)[bash] is not a valid tool, try one of []. I can use the echo command to output the script as markdown code block.
Action: [echo]
Action Input: ` ```bash` (to start the markdown code block)[echo] is not a valid tool, try one of []. I should use the backslash character to escape the special characters.
Action: [echo]
Action Input: ` \` `` (to escape the backtick character)[echo] is not a valid tool, try one of []. I can use the "ls -l" command to list files in long format.
Action: [echo]
Action Input: ` ```bash` (to end the markdown code block)[echo] is not a valid tool, try one of []. I now know the final answer.
Final Answer: ```bash
ls -l
```


> Finished chain.


{'input': 'Write me a shell script to list the files in the current directory. Output the script as markdown code block',
 'output': '```bash\nls -l\n```'}